In [1]:
import tensorflow as tf

当创建一个变量时，你将一个张量作为初始值传入构造函数Variable()。TensorFlow提供了一系列操作符来初始化张量，初始值是常量或是随机值。

注意，所有这些操作符都需要你指定张量的shape。那个形状自动成为变量的shape。变量的shape通常是固定的，但TensorFlow提供了高级的机制来重新调整其行列数

In [2]:
# Create two variables.
weights = tf.Variable(tf.random_normal([784, 200], stddev=0.35),
                      name="weights")
biases = tf.Variable(tf.zeros([200]), name="biases")

变量的初始化必须在模型的其它操作运行之前先明确地完成。最简单的方法就是添加一个给所有变量初始化的操作，并在使用模型之前首先运行那个操作,其实是tf.assign。你或者可以从检查点文件中重新获取变量值,使用tf.initialize_all_variables()添加一个操作对变量做初始化。记得在完全构建好模型并加载之后再运行那个操作。
tf.initialize_all_variables()函数便捷地添加一个op来初始化模型的所有变量。你也可以给它传入一组变量进行初始化

In [4]:
# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Later, when launching the model
with tf.Session() as sess:
  # Run the init operation.
  sess.run(init_op)
  ...
  # Use the model
  ...

### 由另一个变量初始化
你有时候会需要用另一个变量的初始化值给当前变量初始化。由于tf.initialize_all_variables()是并行地初始化所有变量，所以在有这种需求的情况下需要小心。
用其它变量的值初始化一个新的变量时，使用其它变量的initialized_value()属性。你可以直接把已初始化的值作为新变量的初始值，或者把它当做tensor计算得到一个值赋予新变量

In [ ]:
# Create a variable with a random value.
weights = tf.Variable(tf.random_normal([784, 200], stddev=0.35),
                      name="weights")
# Create another variable with the same value as 'weights'.
w2 = tf.Variable(weights.initialized_value(), name="w2")
# Create another variable with twice the value of 'weights'
w_twice = tf.Variable(weights.initialized_value() * 0.2, name="w_twice")

### 保存和加载
最简单的保存和恢复模型的方法是使用tf.train.Saver对象。构造器给graph的所有变量，或是定义在列表里的变量，添加save和restoreops。saver对象提供了方法来运行这些ops，定义检查点文件的读写路径
### 检查点文件
变量存储在二进制文件里，主要包含从变量名到tensor值的映射关系。

当你创建一个Saver对象时，你可以选择性地为检查点文件中的变量挑选变量名。默认情况下，将每个变量Variable.name属性的值
### 保存变量
用tf.train.Saver()创建一个Saver来管理模型中的所有变量

In [ ]:
# Create some variables.
v1 = tf.Variable(..., name="v1")
v2 = tf.Variable(..., name="v2")
...
# Add an op to initialize the variables.
init_op = tf.initialize_all_variables()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, save the
# variables to disk.
with tf.Session() as sess:
  sess.run(init_op)
  # Do some work with the model.
  ..
  # Save the variables to disk.
  save_path = saver.save(sess, "/tmp/model.ckpt") #将变量存至硬盘中
  print "Model saved in file: ", save_path

#### 恢复变量
用同一个Saver对象来恢复变量。注意，当你从文件中恢复变量时，不需要事先对它们做初始化

In [ ]:
# Create some variables.
v1 = tf.Variable(..., name="v1")
v2 = tf.Variable(..., name="v2")
...
# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
  # Restore variables from disk.
  saver.restore(sess, "/tmp/model.ckpt")
  print "Model restored."
  # Do some work with the model
  ...

### 选择存储和恢复哪些变量
如果你不给tf.train.Saver()传入任何参数，那么saver将处理graph中的所有变量。其中每一个变量都以变量创建时传入的名称被保存。

有时候在检查点文件中明确定义变量的名称很有用。举个例子，你也许已经训练得到了一个模型，其中有个变量命名为"weights"，你想把它的值恢复到一个新的变量"params"中。

有时候仅保存和恢复模型的一部分变量很有用。再举个例子，你也许训练得到了一个5层神经网络，现在想训练一个6层的新模型，可以将之前5层模型的参数导入到新模型的前5层中。

你可以通过给tf.train.Saver()构造函数传入Python字典，很容易地定义需要保持的变量及对应名称：键对应使用的名称，值对应被管理的变量。

注意：

如果需要保存和恢复模型变量的不同子集，可以创建任意多个saver对象。同一个变量可被列入多个saver对象中，只有当saver的restore()函数被运行时，它的值才会发生改变。
如果你仅在session开始时恢复模型变量的一个子集，你需要对剩下的变量执行初始化op。详情请见tf.initialize_variables()。

In [ ]:
 Create some variables.
v1 = tf.Variable(..., name="v1")
v2 = tf.Variable(..., name="v2")
...
# Add ops to save and restore only 'v2' using the name "my_v2"
saver = tf.train.Saver({"my_v2": v2})
# Use the saver object normally after that.
...

In [1]:
# Create some variables.
import tensorflow as tf 
v1 = tf.Variable([1,2,3], name="v1_name")
v2 = tf.Variable([4,5,6], name="v2_name")
...
init_op = tf.global_variables_initializer()
# Add ops to save and restore all the variables.
saver = tf.train.Saver([v1,v2])
#变量在文件中保侟的类似于字典是（name:value形式）
# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
  # Restore variables from disk.
  sess.run(init_op)
  save_path = saver.save(sess, "./model_1.ckpt") #将变量存至硬盘中
  print("Model saved in file: ", save_path)

Model saved in file:  ./model_1.ckpt


In [1]:
import tensorflow as tf 
p1 = tf.Variable([7,8,9], name="v1_name")
p2 = tf.Variable([41,51,61], name="v2_name")
init_op = tf.global_variables_initializer()

In [2]:
# Create some variables.

...
# Add ops to save and restore all the variables.
saver = tf.train.Saver([p1])
#根据p1的name去索引，然后将value恢复
# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
  # Restore variables from disk.
  sess.run(init_op)
  saver.restore(sess, "./model_1.ckpt")
  print(sess.run(p1))
  print(sess.run(p2))
  # Do some work with the model
  ...

INFO:tensorflow:Restoring parameters from ./model_1.ckpt
[1 2 3]
[41 51 61]


## 总结
* saver.save()是保存的变量格式是key-valu形式，即文件中是{'varialbe_name':value}
* saver.restore是也用按key(变量名恢复)如上式中只有v1_name于文件中才可以恢复
* 如果只恢复部分变量要初始化
* 多次运行 tf.Variable([7,8,9], name="v1_name") 会生成v1_name_1,v1_name_1....等变量，觉得tf主要是通过name来索引变量
* saver([要存和恢复的变量名] saver中可以指定存为saver([key,vale])而仅是变量本来的name